In [1]:
%cd ..

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3


/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
from pathlib import Path

import pandas as pd
from tqdm import tqdm

from augments import ParagraphGenerator
from utils import create_experiment_file_name

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
DATA_DIR = Path("data")
RESULT_DIR = "experiments"
TARGET_DATA = "train_exp_integration.csv"
EXP_VERSION = "v3.0.1"  # 컨벤션에 따라 새롭게 실험할 경우 버저닝 정보를 기록해주세요.

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"

pg = ParagraphGenerator()
df = pd.read_csv(DATA_DIR / RESULT_DIR / TARGET_DATA)

In [ ]:
target_df = df[df["category"] == "외적 추론"]
target_df.head()

,id,category,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_1_exists,keyword_2_exists,keyword_3_exists,keyword_4_exists,keyword_5_exists,paragraph,question,choices,answer,question_plus
0,generation-for-nlp-1000,외적 추론,서부 이주,오리건 준주,캘리포니아,개척자,공화국,False,True,True,False,True,"""넓고 무한한 미래는 위대한 미국의 시대가 될 것이다. 많은 나라로 구성된 나라는 ...",다음 서부 이주의 변화 중 1840년대에 일어났을 가능성이 가장 높은 것은 무엇입니까?,"['오리건 준주로 향하는 개척자 수는 감소한 반면, 캘리포니아로 향하는 개척자 수는...",1,NaN
1,generation-for-nlp-1001,외적 추론,기혼 여성,재산 보호,남편의 처분,법률,미혼 여성,False,False,False,True,True,"""기혼 여성의 재산을 보다 효과적으로 보호하기 위한 법률:\n""§1. 앞으로 결혼할...",이 지문에 나타난 정보는 무엇입니까?,"['투표권에 대한 접근권을 둘러싼 토론', '결혼법에서 연방정부의 역할을 둘러싼 토...",4,NaN
2,generation-for-nlp-1003,외적 추론,독재자,유엔,신세계 질서,테러,평화,True,True,True,True,True,명백히 독재자 하나가 더는 침략에 맞서는 유엔 공동행동을 방해할 목적으로 동서 대결...,"부시 대통령의 ""신세계 질서""와 가장 직접적으로 닮은 것은 다음 중 무엇입니까?","['먼로 독트린', '시어도어 루즈벨트의 ""곤봉정책""', '우드로 윌슨의 14개조'...",3,NaN
3,generation-for-nlp-1004,외적 추론,자유,권리,폭정,해밀턴,젠거 사건,True,True,False,True,False,"상처를 입으면 불평할 권리가 있음은 모든 자유인이 주장하는 당연한 일이자, 특권이자...",젠거 사건에서 해밀턴의 승소가 가장 직접적으로 반영하는 것은 다음 중 무엇입니까?,"['영국으로부터 독립하려는 미국의 열망', '계몽주의 이상에 대한 미국의 거부', ...",3,NaN
4,generation-for-nlp-1005,외적 추론,포퓰리즘,정치운동,헌법,개혁,진보주의,True,True,True,True,True,"""우리는 25년 넘게 두 거대 정당이 권력을 얻고 빼앗으려 드는 동안 고통받는 자들...",포퓰리즘당이 제시한 많은 목표를 가장 밀접하게 공유하는 정치운동은 다음 중 무엇입니까?,"['1900년대 초반 진보주의', '1980년대 신보수주의', '1700년대 후반 ...",1,NaN


In [ ]:
sampled_df = target_df.sample(20, random_state=1004)

In [ ]:
def filter_keywords_by_exists(row):
    """
    단일 행에서 `exists` 열이 True인 키워드만 필터링합니다.

    Args:
        row (pd.Series): keyword 및 keyword_x_exists 열을 포함한 DataFrame의 단일 행.

    Returns:
        list: 해당 `exists` 열이 True인 키워드의 리스트.
    """
    # keyword와 exists 열 쌍 정의
    keyword_columns = [f"keyword_{i}" for i in range(1, 6)]
    exists_columns = [f"keyword_{i}_exists" for i in range(1, 6)]

    # exists 열을 기준으로 키워드 필터링
    filtered_keywords = [row[keyword] for keyword, exists in zip(keyword_columns, exists_columns) if row[exists]]

    return filtered_keywords


tqdm.pandas()
sampled_df["generated_paragraph"] = sampled_df.progress_apply(
    lambda row: pg.generate(
        "\n".join([row[PARAGRAPH], str(row[QUESTION_PLUS]), row[QUESTION], row[CHOICES]]),
        filter_keywords_by_exists(row),
    ),
    axis=1,
)
sampled_df.head()

100%|██████████| 20/20 [03:46<00:00, 11.33s/it]


,id,category,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_1_exists,keyword_2_exists,keyword_3_exists,keyword_4_exists,keyword_5_exists,paragraph,question,choices,answer,question_plus,generated_paragraph
1615,generation-for-nlp-982,외적 추론,링컨,노예제,노예해방 선언,남북전쟁,인종적 견해,True,True,True,True,False,"""링컨은 노예제를 강력하게 반대했으나 노예제 폐지론자나 급진 공화당원도 아니었고 자...",링컨 대통령의 노예해방 선언으로 남북전쟁의 과정이 어떻게 변경되었습니까?,['선언문으로 인해 남부연합이 자신들의 대의가 무익함을 깨달아 전쟁은 신속하게 종결...,4,NaN,링컨 대통령의 노예해방 선언은 미국 역사에서 중요한 전환점을 나타낸다. 이 선언은 ...
1290,generation-for-nlp-539,외적 추론,굶주림,병든 주민,뿌리,성직자,흑사병,True,False,True,True,True,"450명의 병든 주민 중 200명은 살릴 수 없었으며, 우리는 그들이 길가에 누워 ...",다음 중 위에 언급된 굶주림 문제를 종식하는 데 가장 큰 역할을 한 것은 무엇입니까?,"['17세기 유럽의 흑사병 종식', '17세기 농업혁명을 통한 휴경지 소멸', '사...",2,NaN,"굶주림은 인류가 직면한 가장 심각한 문제 중 하나로, 이는 단순히 식량의 부족을 넘..."
65,generation-for-nlp-1098,외적 추론,필리핀,정치적 개입,스페인-미국 전쟁,자치권,비버리지,True,False,True,True,False,"""반대파는 우리가 국민의 동의 없이 통치해서는 안 된다고 말한다. 나는 대답한다. ...",미국이 필리핀에 정치적으로 개입하게 된 군사적 충돌은?,"['멕시코-미국 전쟁', '제2차 세계대전', '스페인-미국 전쟁', '복서 반란']",3,NaN,"필리핀 공화국은 동남아시아의 섬나라로, 다양한 민족과 문화가 공존하는 지역이다. 필..."
1288,generation-for-nlp-537,외적 추론,경험적,추론,공리,베이컨,일반화,False,True,True,True,True,"“11. 현재의 과학이 효과의 발견에 쓸모가 없듯, 현재의 논리 체계도 과학의 발견...",위 글에서 설명하고 있는 경험적 추론은 1800년대까지 어떠한 변화를 겪었습니까?,"['의미가 없어질 정도로 약화되었다.', '유럽 문화의 핵심 원칙이 되었다.', '...",2,NaN,추론은 우리가 이미 알고 있는 정보나 확인된 사실을 바탕으로 새로운 결론을 도출하는...
130,generation-for-nlp-1208,외적 추론,예루살렘,십자군 전쟁,고문,교회,샤를마뉴,True,True,True,True,True,"""예루살렘과 콘스탄티노플 시 경계로부터 끔찍한 이야기가 흘러나와 우리의 귀까지 들어...",다음 중 위 글이 가장 잘 이해되는 맥락은 무엇입니까?,"['몽골의 중동 침략', '종교개혁', '대항해 시대', '십자군 전쟁']",4,NaN,"예루살렘은 역사적으로 기독교, 유대교, 이슬람교의 성지로 여겨지며, 이 지역의 정치..."


In [ ]:
result_file_name = create_experiment_file_name(target_file=TARGET_DATA, version=EXP_VERSION)
result_file_path = DATA_DIR / RESULT_DIR / result_file_name
sampled_df.to_csv(result_file_path, index=False)